<a href="https://colab.research.google.com/github/oliverrmaa/data-wrangling-springboard/blob/main/notebooks/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection

In [ ]:
# Installations:
!pip install tabula-py

# Imports:
import requests
import json
from bs4 import BeautifulSoup
from tabula.io import read_pdf
import pandas as pd

# Google Drive mount authentication
from google.colab import drive
drive.mount('/content/drive')

# 1. Fetch Data From API

## 1.1 Exchange Rate Demo

https://exchangeratesapi.io

In [ ]:
API_KEY = "7d621c8860a896ffe2d06c2f27317372"
URL = "http://api.exchangeratesapi.io/v1/latest?access_key={}".format(API_KEY)
 
response = requests.get(URL)
print(response)
text = response.text
json_data = json.loads(text)

In [ ]:
print(json_data)
print(json_data.keys())

In [ ]:
df = pd.DataFrame({"rate": json_data['rates']})
df.head()

In [ ]:
df = df.reset_index(inplace=False)
df.head()

In [ ]:
df = df.rename(columns={'index': 'currency'})
df.head()

In [ ]:
df['base_currency'] = json_data['base']
df['date_accessed'] = json_data['date']
df.head()

## 1.2 Exercise: How Many People Are In Space?

Let's use an open API so we do not need to worry about authentication. Check how many people are currently in space using this open API. Be sure to reshape
the resulting JSON into a proper DataFrame. For the URL, please use:
`"http://api.open-notify.org/astros.json"`

In [ ]:
# YOUR CODE GOES HERE

# 2. Fetch Data Via Parse HTML

## 2.1 Video Game Music Demo

We will parse a webpage full or links (https://www.vgmusic.com/music/console/nintendo/nes/).
 
This would be of interest if we were harvesting data stored in links from a web-based source. There are important background information on html:
- `<a>`is known as an anchor element in HTML, we usually search for these elements when parsing for links. (`https://developer.mozilla.org/en-US/docs/Web/HTML/Element/a`)
- `href` is an attribue of `<a>` which creates the hyperlink

In [ ]:
URL = 'https://www.vgmusic.com/music/console/nintendo/nes/'
response = requests.get(URL)
text = response.text
soup = BeautifulSoup(text, 'html.parser')

In [ ]:
soup.find_all("a")[0:10]

In [ ]:
len(soup.find_all("a"))

In [ ]:
href_list = [link.get("href") for link in soup.find_all("a")]

# code without using list comprehension:
# href_list = []
# for link in soup.find_all("a"):
#   href_list.append(link.get("href"))

In [ ]:
df = pd.DataFrame({"song_list": href_list, "base_url": URL})

In [ ]:
len(df.song_list)

In [ ]:
aux = df[df["song_list"].str.contains(".mid", na=False)]
aux.head()

In [ ]:
aux.reset_index(inplace=True)
aux.head()

In [ ]:
# axis=1 is for columns 
aux = aux.drop('index', axis=1)

In [ ]:
aux['full_path'] = aux['base_url'] + aux["song_list"]
aux.head()

In [ ]:
len(aux)

In [ ]:
aux.full_path[0]

## 2.2 Exercise: Google Scholar Link Extraction

Let's practice link extraction from html. For the following google scholar link,
extract every link that leads to an academic article. Be sure to put the results into a nice list or DataFrame showing just the necessary links.

Please use the following google scholar link: https://scholar.google.ca/scholar?hl=en&as_sdt=0%2C5&q=astroids&btnG=

In [ ]:
# YOUR CODE GOES HERE

# 3. Fetch Data From PDF

https://camelot-py.readthedocs.io/en/master/_static/pdf/foo.pdf

In [ ]:
URL = "https://camelot-py.readthedocs.io/en/master/_static/pdf/foo.pdf"
df_list = read_pdf(URL, output_format="dataframe", pages="all")

In [ ]:
df = df_list[0]
df.head()

In [ ]:
df.columns

In [ ]:
df['decrease_accel'] = df["Percent Fuel Savings"].str.split(" ").str[0]
df['eliminate_stops'] = df["Percent Fuel Savings"].str.split(" ").str[1]
df.head()

In [ ]:
df = df[3:7]
df = df.rename(columns={
    "Unnamed: 0": "cycle_name", 
    "Unnamed: 1": "KI",
    "Unnamed: 2": "distance",
    "Unnamed: 3": "improved_speed",
    "Unnamed: 5": "decrease_idle"
  })

In [ ]:
df = df[["cycle_name", "KI", "distance", "improved_speed", "decrease_accel", "eliminate_stops", "decrease_idle"]]
df.head()

# 4. Fetch Data From CSV / Save Data To CSV 

https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv

In [ ]:
URL = "https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv"
df = pd.read_csv(URL)

In [ ]:
df.head()

In [ ]:
PATH = "/content/drive/MyDrive/00_temp/countries.csv"
df.to_csv(PATH, index=False)